In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from joblib import dump

C:\Users\Alok Yadav\anaconda3\envs\technocolab\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
# Create your models here.
data = pd.read_csv("C:/Users/Alok Yadav/Desktop/TechnoColab/Main_Project/mi_processed_data.csv")
X = data.drop(['status', 'isClosed'], axis=1)  # drop the target variable and the newly created binary target
y_binary = data['isClosed']  # binary target
y_multiclass = data['status']  # multiclass target

In [3]:
X_train, X_test, y_multiclass_train, y_multiclass_test = train_test_split(X, y_multiclass, test_size=0.25, random_state=42)

In [4]:
# # Extracting specific columns for different target variables:

# yStatus_train = y_train.iloc[:,0]
# yClosed_train = y_train.iloc[:,1]

# yStatus_test = y_test.iloc[:,0]
# yClosed_test = y_test.iloc[:,1]

In [5]:
# stdscaler = StandardScaler()
# minmaxscaler = MinMaxScaler()
# pca = PCA(n_components=0.9)
# smote = SMOTE(random_state=42)

In [6]:
# XGB = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
# LGBM = LGBMClassifier()
# NB = MultinomialNB()

In [7]:
# model_xgb = ImbPipeline([
#     ('stdscaler', stdscaler),
#     ('pca', pca),
#     ('smote', smote),
#     ('classifier', XGB)
# ])

# # Fit the model:
# model_xgb.fit(X_train, yClosed_train)

In [8]:
# model_lgbm = ImbPipeline([
#     ('stdscaler', stdscaler),
#     ('pca', pca),
#     ('smote', smote),
#     ('classifier', LGBM)
# ])

# # Fit the model:
# model_lgbm.fit(X_train, yClosed_train)

In [9]:
# model_nb = ImbPipeline([
#     ('minmaxscaler', minmaxscaler),
#     ('smote', smote),
#     ('classifier', NB)
# ])

# # Fit the model:
# model_nb.fit(X_train, yStatus_train)

In [10]:
# class CustomPipeline(BaseEstimator, ClassifierMixin):
#     def __init__(self, binary_model_1, binary_model_2, multiclass_model, label_encoder=None):
#         self.binary_model_1 = binary_model_1
#         self.binary_model_2 = binary_model_2
#         self.multiclass_model = multiclass_model
#         self.label_encoder = label_encoder

#     def fit(self, X, y_binary, y_multiclass):
#         # Fit binary models on all data
#         self.binary_model_1.fit(X, y_binary)
#         self.binary_model_2.fit(X, y_binary)

#         # Fit multiclass model on filtered subsets for both sets of conditions
#         multiclass_indices = (self.binary_model_1.predict(X) == 1) & (self.binary_model_2.predict(X) == 1)
#         acquired_closed_indices = (self.binary_model_1.predict(X) == 0) | (self.binary_model_2.predict(X) == 0)

#         # Perhaps check the actual labels being passed
#         print("Labels for multiclass training:", y_multiclass[multiclass_indices | acquired_closed_indices])

#         self.multiclass_model.fit(X[multiclass_indices | acquired_closed_indices], y_multiclass[multiclass_indices | acquired_closed_indices])
#         return self

#     def predict(self, X):
#         pred_1 = self.binary_model_1.predict(X)
#         pred_2 = self.binary_model_2.predict(X)
#         final_predictions = np.empty(X.shape[0], dtype=int)  # ensure dtype is int for consistency

#         for i in range(X.shape[0]):
#             if pred_1[i] == 1 and pred_2[i] == 1:
#                 # Predict 'operating' or 'IPO'
#                 raw_pred = self.multiclass_model.predict(X[i:i+1])[0]
#             else:
#                 # Predict 'acquired' or 'closed'
#                 raw_pred = self.multiclass_model.predict(X[i:i+1])[0]

#             # Ensure predictions are handled as integers
#             final_predictions[i] = int(raw_pred)  # Make sure predictions are integers

#         return final_predictions

In [11]:
# # Define all possible labels explicitly
# all_labels = ['operating', 'ipo', 'acquired', 'closed']  # Add all potential labels
# label_encoder = LabelEncoder()
# label_encoder.fit(all_labels)

# # Encode the multiclass labels using the label encoder
# yStatus_train_encoded = label_encoder.transform(yStatus_train)
# yStatus_test_encoded = label_encoder.transform(yStatus_test)

In [12]:
# combined_pipeline = CustomPipeline(
#     binary_model_1=model_xgb,
#     binary_model_2=model_lgbm,
#     multiclass_model=model_nb,
#     label_encoder=label_encoder
# )

# # Fit the pipeline using the encoded multiclass labels and binary labels
# combined_pipeline.fit(X_train, yClosed_train, yStatus_train_encoded)

In [13]:
# # Define the base path for your models
# base_path = 'C:/Users/Alok Yadav/Desktop/TechnoColab/Main_Project/WebApp/myproject/mlmodel/models/'

# # Save the XGB model
# dump(model_xgb, base_path + "model_xgb.joblib")

# # Save the LGBM model
# dump(model_lgbm, base_path + "model_lgbm.joblib")

# # Save the NB model
# dump(model_nb, base_path + "model_nb.joblib")

# # Save the combined pipeline
# dump(combined_pipeline, base_path + "combined_pipeline.joblib")

In [14]:
from sklearn.base import BaseEstimator, ClassifierMixin

class CustomPipeline(BaseEstimator, ClassifierMixin):
    def __init__(self):
        # Initialize the binary and multiclass models
        self.binary_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
        self.model_open = MultinomialNB()
        self.model_closed = MultinomialNB()
        self.scaler = MinMaxScaler()  # Scaler for the input features
        self.encoder = LabelEncoder()  # Encoder for the target labels
        self.smote = SMOTE(random_state=42)  # SMOTE for handling class imbalance

    def fit(self, X, y):
        # Fit the scaler to the data
        X_scaled = self.scaler.fit_transform(X)

        # Encode and SMOTE transform the multiclass target
        y_encoded = self.encoder.fit_transform(y)
        X_smote, y_smote = self.smote.fit_resample(X_scaled, y_encoded)

        # Fit the binary model
        y_binary = (y_smote > 1)  # Regenerate binary targets: 'operating'/'ipo' as 1, 'acquired'/'closed' as 0
        self.binary_model.fit(X_smote, y_binary)

        # Prepare data for multiclass models based on binary target
        open_mask = y_binary == 1
        closed_mask = y_binary == 0

        # Train the 'open' model
        self.model_open.fit(X_smote[open_mask], y_smote[open_mask])

        # Train the 'closed' model
        self.model_closed.fit(X_smote[closed_mask], y_smote[closed_mask])

        return self

    def predict(self, X):
        # Scale the features
        X_scaled = self.scaler.transform(X)

        # Predict using the binary model
        binary_predictions = self.binary_model.predict(X_scaled)

        # Determine the final predictions based on the binary outcome
        final_predictions = []
        for pred, features in zip(binary_predictions, X_scaled):
            if pred == 1:
                final_pred = self.encoder.inverse_transform(self.model_open.predict([features]))[0]
            else:
                final_pred = self.encoder.inverse_transform(self.model_closed.predict([features]))[0]
            final_predictions.append(final_pred)

        return final_predictions

In [15]:
# Create and fit the new pipeline with SMOTE
combined_pipeline = CustomPipeline()
combined_pipeline.fit(X_train, y_multiclass_train)

CustomPipeline()

In [17]:
smote_predictions = combined_pipeline.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_multiclass_test, smote_predictions))
print("Classification Report:\n", classification_report(y_multiclass_test, smote_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_multiclass_test, smote_predictions))

Accuracy: 0.6500835840855901
Classification Report:
               precision    recall  f1-score   support

    acquired       0.82      0.75      0.78       883
      closed       0.51      0.61      0.56       380
         ipo       0.02      0.81      0.03       104
   operating       1.00      0.64      0.78     13588

    accuracy                           0.65     14955
   macro avg       0.59      0.70      0.54     14955
weighted avg       0.97      0.65      0.77     14955

Confusion Matrix:
 [[ 661  221    0    1]
 [ 143  233    2    2]
 [   0    0   84   20]
 [   1    3 4840 8744]]


In [20]:
# # Define the base path for your models
# base_path = 'C:/Users/Alok Yadav/Desktop/TechnoColab/Main_Project/WebApp/myproject/mlmodel/models/'

# # Save the combined pipeline
# dump(combined_pipeline, base_path + "combined_pipeline.joblib")

['C:/Users/Alok Yadav/Desktop/TechnoColab/Main_Project/WebApp/myproject/mlmodel/models/combined_pipeline.joblib']

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.preprocessing import MinMaxScaler

# # Update the numerical transformer to ensure non-negative values for Naive Bayes
# numerical_transformer_nb = MinMaxScaler()

# # Combine transformers into a preprocessor for Naive Bayes
# preprocessor_nb = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer_nb, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Define the Multiclass classifier in a pipeline
# pipeline_nb = Pipeline(steps=[
#     ('preprocessor', preprocessor_nb),
#     ('classifier', MultinomialNB())
# ])

# # Fit the Multinomial Naive Bayes model
# pipeline_nb.fit(X_train_multi, y_train_multi)

# 'Multinomial Naive Bayes model is fitted and ready for predictions.'
